In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
from scipy.fft import fft
from scipy.stats import entropy,iqr

In [3]:
df = pd.read_csv("./অ_preprocessed_1.csv")

In [4]:
#extracting x,y,z norm

x,y,z = df["x_norm"],df["y_norm"],df["z_norm"]

## Computing RMS
### 📌 What it is:
### Measures the overall intensity of movement along each axis.

### 🧠 Why it helps:
### Some characters require strong, fast strokes (e.g. “র”), others are gentler (e.g. “ই”).
### RMS tells the model how “powerful” the movement was.

In [5]:
def compute_rms(series):
    return np.sqrt(np.mean(series**2))

## Computing Zero-Crossing
### 📌 What it is:
### Counts how often the signal changes direction (crosses the zero line).

### 🧠 Why it helps:
###  drawing “জ” — moving back and forth more → high ZCR.
###  “|” (just a line) → low ZCR.
### It tells the model how “zigzaggy” the stroke is.



In [6]:
def compute_zcr(series):
    return np.mean(np.diff(np.sign(series)) != 0)

## Entropy
### 📌 What it is:
### Measures randomness or complexity in the movement.

### 🧠 Why it helps:
### Characters like “ঘ” have loops and complexity → higher entropy.
### Simple letters like “অ” have more predictable strokes → lower entropy.
### Entropy helps the model tell simple vs complex strokes apart.

In [7]:
def compute_entropy(series,bins=30):
    hist,_ = np.histogram(series, bins = bins, density = True)
    hist = hist[hist>0]
    return entropy(hist)

## Extract Features

In [8]:
features={
    "RMS_x" : compute_rms(x),
    "RMS_y" : compute_rms(y),
    "RMS_z" : compute_rms(z),
    "Entropy_x" : compute_entropy(x),
    "Entropy_y" : compute_entropy(y),
    "Entropy_z" : compute_entropy(z),
    "ZCR_x" : compute_zcr(x),
    "ZCR_y" : compute_zcr(y),
    "ZCR_z" : compute_zcr(z),
    "IQR_x": iqr(x),
    "IQR_y": iqr(y),
    "IQR_z": iqr(z),
    "Corr_xy": np.corrcoef(x, y)[0, 1],
    "Corr_yz": np.corrcoef(y, z)[0, 1],
    "Corr_zx": np.corrcoef(z, x)[0, 1]
}

## Converting into 1 Dataframe

In [9]:
features_df = pd.DataFrame([features])

In [10]:
features_df

,RMS_x,RMS_y,RMS_z,Entropy_x,Entropy_y,Entropy_z,ZCR_x,ZCR_y,ZCR_z,IQR_x,IQR_y,IQR_z,Corr_xy,Corr_yz,Corr_zx
0,1.0,1.0,1.0,2.637917,2.839853,2.264464,0.006272,0.007168,0.021505,0.830156,1.072727,0.258812,-0.189886,0.016501,0.589275


In [11]:
x, y, z = df['x_norm'], df['y_norm'], df['z_norm']
N = len(x)

# -- FFT Feature Extractor --
def extract_fft_features(signal, N):
    fft_vals = np.abs(fft(signal))[:N // 2]
    fft_freqs = np.fft.fftfreq(N)[:N // 2]

    if np.sum(fft_vals) == 0:
        return {
            "spectral_entropy": 0,
            "spectral_energy": 0,
            "spectral_centroid": 0,
            "dominant_frequency": 0
        }

    fft_norm = fft_vals / np.sum(fft_vals)

    return {
        "spectral_entropy": entropy(fft_norm),
        "spectral_energy": np.sum(fft_vals ** 2) / N,
        "spectral_centroid": np.sum(fft_freqs * fft_vals) / np.sum(fft_vals),
        "dominant_frequency": fft_freqs[np.argmax(fft_vals)]
    }

# -- FFT features for each axis --
x_fft = extract_fft_features(x, N)
y_fft = extract_fft_features(y, N)
z_fft = extract_fft_features(z, N)

# -- Final combined feature dict --
features = {
    # FFT features
    "x_spectral_entropy": x_fft["spectral_entropy"],
    "y_spectral_entropy": y_fft["spectral_entropy"],
    "z_spectral_entropy": z_fft["spectral_entropy"],

    "x_spectral_energy": x_fft["spectral_energy"],
    "y_spectral_energy": y_fft["spectral_energy"],
    "z_spectral_energy": z_fft["spectral_energy"],

    "x_spectral_centroid": x_fft["spectral_centroid"],
    "y_spectral_centroid": y_fft["spectral_centroid"],
    "z_spectral_centroid": z_fft["spectral_centroid"],

    "x_dominant_freq": x_fft["dominant_frequency"],
    "y_dominant_freq": y_fft["dominant_frequency"],
    "z_dominant_freq": z_fft["dominant_frequency"],

    
    "RMS_x": compute_rms(x),
    "RMS_y": compute_rms(y),
    "RMS_z": compute_rms(z),

    "Entropy_x": compute_entropy(x),
    "Entropy_y": compute_entropy(y),
    "Entropy_z": compute_entropy(z),

    "ZCR_x": compute_zcr(x),
    "ZCR_y": compute_zcr(y),
    "ZCR_z": compute_zcr(z),

    "IQR_x": iqr(x),
    "IQR_y": iqr(y),
    "IQR_z": iqr(z),

    "Corr_xy": np.corrcoef(x, y)[0, 1],
    "Corr_yz": np.corrcoef(y, z)[0, 1],
    "Corr_zx": np.corrcoef(z, x)[0, 1]
}


features_df = pd.DataFrame([features])
